## Base Model 

Youth criminalization

This notebook defines three agent types: `YouthAgent`, `HouseholdAgent`, and `SchoolAgent`,
and includes a small example simulation to explore youth crime involvement dynamics.

In [ ]:
# Very basic Mesa model for youth criminalization (minimal attributes & logic)
import random
from collections import defaultdict
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import matplotlib.pyplot as plt
import pandas as pd

class YouthAgent(Agent):
    """Simplified youth agent with a peer_group (friends)."""
    def __init__(self, unique_id, model, age, neighbourhood_id, peer_group):
        super().__init__(unique_id, model)
        self.age = age
        self.neighbourhood_id = neighbourhood_id
        self.peer_group = peer_group
        self.school_status = random.choice(['enrolled', 'dropped_out'])
        self.attendance_rate = 1.0 if self.school_status == 'enrolled' else 0.4
        self.risk_propensity = random.random()  # 0-1
        self.crime_involvement = 'none'  # none, minor, serious
        self.criminal_record = 0

    def step(self):
        # Peer influence comes from the agent's peer group (friends)
        peer_influence = self.model.peer_group_criminality.get(self.peer_group, 0.0)
        # Simple probability combining individual risk and peer influence
        prob = 0.02 + 0.3 * self.risk_propensity + 0.4 * peer_influence
        if random.random() < prob:
            # Mostly minor offences, sometimes serious
            self.crime_involvement = 'minor' if random.random() < 0.7 else 'serious'
            if random.random() < 0.2:
                self.criminal_record += 1
        else:
            self.crime_involvement = 'none'

class YouthModel(Model):
    """Minimal model: youth agents are assigned to peer groups (friends)."""
    def __init__(self, n_youth=100, n_neighbourhoods=5, groups_per_neighbourhood=2, seed=None):
        if seed is not None:
            random.seed(seed)
        self.schedule = RandomActivation(self)
        # peer groups: group id -> list of agents
        self.peer_groups = defaultdict(list)
        # peer group criminality: group id -> fraction offending
        self.peer_group_criminality = {}

        # Create peer group ids (simple scheme: numbered groups)
        n_groups = max(1, n_neighbourhoods * groups_per_neighbourhood)
        group_ids = list(range(n_groups))

        # Create youth agents and assign simple neighbourhoods and peer groups
        for i in range(n_youth):
            age = random.randint(12, 24)
            neighbourhood = i % n_neighbourhoods
            # assign peer group (biased by neighbourhood if desired)
            pg = random.choice(group_ids)
            a = YouthAgent(i, self, age, neighbourhood, peer_group=pg)
            self.schedule.add(a)
            self.peer_groups[pg].append(a)

        # Data collector to track counts each step
        self.datacollector = DataCollector(
            model_reporters={
                'none': lambda m: sum(1 for a in m.schedule.agents if a.crime_involvement == 'none'),
                'minor': lambda m: sum(1 for a in m.schedule.agents if a.crime_involvement == 'minor'),
                'serious': lambda m: sum(1 for a in m.schedule.agents if a.crime_involvement == 'serious')
            }
        )

    def step(self):
        # compute simple peer-group criminality (fraction offending per group)
        for gid, members in self.peer_groups.items():
            if not members:
                self.peer_group_criminality[gid] = 0.0
                continue
            offending = sum(1 for m in members if m.crime_involvement != 'none')
            self.peer_group_criminality[gid] = offending / len(members)

        # step agents and collect data
        self.schedule.step()
        self.datacollector.collect(self)

# Example quick run (works interactively in the notebook)
if __name__ == '__main__':
    m = YouthModel(n_youth=150, n_neighbourhoods=5, groups_per_neighbourhood=2, seed=1)
    for i in range(12):
        m.step()
    df = m.datacollector.get_model_vars_dataframe()
    print(df.tail())
    df[['none', 'minor', 'serious']].plot(title='Crime involvement over time')
    plt.xlabel('Step')
    plt.ylabel('Number of youth')
    plt.show()

# In the notebook: create `m = YouthModel(...)`, run `m.step()` repeatedly,
# then inspect `m.datacollector.get_model_vars_dataframe()` or individual agents via `m.schedule.agents`.
